In [1]:
import numpy as np
import scipy.optimize as optimize

In [2]:
combo = []
for k1 in range(2, 12):
    for m in range(k1+1, k1+9):
        for n in range(k1+1, k1+9):
            k2 = (k1*(m+n-1)+2*n)/(2*n+m-1)
            if (k2%1==0):
                if [k1, k2, [m, n]] not in combo: 
                    combo.append([k1, k2, [m, n]])
print(combo)

[[2, 2.0, [3, 3]], [2, 2.0, [3, 4]], [2, 2.0, [3, 5]], [2, 2.0, [3, 6]], [2, 2.0, [3, 7]], [2, 2.0, [3, 8]], [2, 2.0, [3, 9]], [2, 2.0, [3, 10]], [2, 2.0, [4, 3]], [2, 2.0, [4, 4]], [2, 2.0, [4, 5]], [2, 2.0, [4, 6]], [2, 2.0, [4, 7]], [2, 2.0, [4, 8]], [2, 2.0, [4, 9]], [2, 2.0, [4, 10]], [2, 2.0, [5, 3]], [2, 2.0, [5, 4]], [2, 2.0, [5, 5]], [2, 2.0, [5, 6]], [2, 2.0, [5, 7]], [2, 2.0, [5, 8]], [2, 2.0, [5, 9]], [2, 2.0, [5, 10]], [2, 2.0, [6, 3]], [2, 2.0, [6, 4]], [2, 2.0, [6, 5]], [2, 2.0, [6, 6]], [2, 2.0, [6, 7]], [2, 2.0, [6, 8]], [2, 2.0, [6, 9]], [2, 2.0, [6, 10]], [2, 2.0, [7, 3]], [2, 2.0, [7, 4]], [2, 2.0, [7, 5]], [2, 2.0, [7, 6]], [2, 2.0, [7, 7]], [2, 2.0, [7, 8]], [2, 2.0, [7, 9]], [2, 2.0, [7, 10]], [2, 2.0, [8, 3]], [2, 2.0, [8, 4]], [2, 2.0, [8, 5]], [2, 2.0, [8, 6]], [2, 2.0, [8, 7]], [2, 2.0, [8, 8]], [2, 2.0, [8, 9]], [2, 2.0, [8, 10]], [2, 2.0, [9, 3]], [2, 2.0, [9, 4]], [2, 2.0, [9, 5]], [2, 2.0, [9, 6]], [2, 2.0, [9, 7]], [2, 2.0, [9, 8]], [2, 2.0, [9, 9]], [2,

In [3]:
npc = np.array(combo, dtype=object)
mn_comb = npc[npc[:,0]==5][:,2]
mn_comb

array([list([7, 6]), list([8, 7]), list([9, 8]), list([10, 9]),
       list([11, 10]), list([12, 11]), list([13, 12])], dtype=object)

In [4]:
""" Optimization Algorithm """
""" New Matrix """
def newMat(x, Vt, k):
  V_new = np.zeros((Vt.shape), dtype=np.cfloat)
  if k==2:
    V_new[0] = np.cos(x[0])
    V_new[1] = (np.sin(x[0])) * np.exp(1j*x[1])
  elif k==3:
    V_new[0] = np.cos(x[0])
    V_new[1] = (np.sin(x[0])) * (np.cos(x[1])) * np.exp(1j*x[2])
    V_new[2] = (np.sin(x[0])) * (np.sin(x[1])) * np.exp(1j*x[3])
  else:
    V_new[0] = (np.cos(x[0])) * (np.cos(x[1]))
    V_new[1] = (np.cos(x[0])) * (np.sin(x[1])) * np.exp(1j*x[3])
    V_new[2] = (np.sin(x[0])) * (np.cos(x[2])) * np.exp(1j*x[4])
    V_new[3] = (np.sin(x[0])) * (np.sin(x[2])) * np.exp(1j*x[5])
  return V_new


""" Cost Function """
def costFn(x, Ut, Vt, A, k):
    V_new = newMat(x, Vt, k)
    Bp = np.dot(Ut, V_new) 
    loss = np.linalg.norm(A - Bp*np.conjugate(Bp))
    return (loss)


In [5]:
def calcResults(k, k_new):
    final_res = []
    for m, n in mn_comb:
        print ("m = ",m,", n = ",n)
        res = np.zeros((100,3))
        for i in range(100):
            A = np.random.rand(m, n)
            A = A/A.sum(axis=0)         # Optimize column-wise

            #Classic Truncated SVD
            U, L, V = np.linalg.svd(A, full_matrices=False)
            Ut = U[:, :k]
            Vt = V[:k]
            Lt = L[:k]
            At = np.dot(np.dot(Ut,np.diag(Lt)), Vt)
            res[i][0] = (np.linalg.norm(A - At))


            # Complex SVD
            B = np.sqrt(A)
            U, L, V = np.linalg.svd(B, full_matrices=False)
            # Complex SVD with k
            if (k<=4):                                      # Skip when k>4
                Ut = U[:, :k]
                Vt = V[:k]
                Lt = L[:k]
                initial_guess = np.ones((2*n*(k-1),), dtype=np.longdouble)
                V_new = np.zeros(Vt.shape, dtype=np.cfloat)
                for col in range(Vt.shape[1]):
                    result = optimize.minimize(fun=costFn, x0=initial_guess, args=(Ut,Vt[:, col],A[:,col],k),
                                            tol=1e-7, method='Nelder-Mead', options={'maxiter':1e+10})
                    V_new[:,col] = newMat(result.x, Vt[:, col], k)
                Bp = np.dot(Ut, V_new)  
                res[i][1] = (np.linalg.norm(A - np.conjugate(Bp)*Bp))
            else:
                res[i][1] = (np.linalg.norm(A - At))

            # Complex SVD with k_new
            Ut = U[:, :k_new]
            Vt = V[:k_new]
            Lt = L[:k_new]
            initial_guess = np.ones((2*n*(k_new-1),), dtype=np.longdouble)
            V_new = np.zeros(Vt.shape, dtype=np.cfloat)
            for col in range(Vt.shape[1]):
                result = optimize.minimize(fun=costFn, x0=initial_guess, args=(Ut,Vt[:, col],A[:,col],k_new),
                                        tol=1e-7, method='Nelder-Mead', options={'maxiter':1e+10})
                V_new[:,col] = newMat(result.x, Vt[:, col], k_new)
            Bp = np.dot(Ut, V_new)  
            res[i][2] = (np.linalg.norm(A - np.conjugate(Bp)*Bp))


            if i%10==0: print(i, end=' ')
        print('\n')
        final_res.append([m, n, *res.mean(axis=0)])
    return final_res

In [6]:
res = calcResults(k=5, k_new=4)

m =  7 , n =  6
0 10 20 30 40 50 60 70 80 90 

m =  8 , n =  7
0 10 20 30 40 50 60 70 80 90 

m =  9 , n =  8
0 10 20 30 40 50 60 70 80 90 

m =  10 , n =  9
0 10 20 30 40 50 60 70 80 90 

m =  11 , n =  10
0 10 20 30 40 50 60 70 80 90 

m =  12 , n =  11
0 10 20 30 40 50 60 70 80 90 

m =  13 , n =  12
0 10 20 30 40 50 60 70 80 90 



In [7]:
res

[[7, 6, 0.04655580837736055, 0.04655580837736055, 0.06228847265417983],
 [8, 7, 0.09836315509265871, 0.09836315509265871, 0.11267007269478763],
 [9, 8, 0.13731053269857907, 0.13731053269857907, 0.1563131642443652],
 [10, 9, 0.175228570350702, 0.175228570350702, 0.19378047645682062],
 [11, 10, 0.20155311640995482, 0.20155311640995482, 0.22013717339173217],
 [12, 11, 0.2321292481889817, 0.2321292481889817, 0.24812067115047015],
 [13, 12, 0.25387064792161373, 0.25387064792161373, 0.26967171803785606]]